In [12]:
#LOAD LIBRARIES AND DECLARE FUNCTIONS
import os
import re
import pickle
from Bio import Entrez
from bioservices.kegg import KEGG
os.chdir('/Users/OmarAlvaS/Documents/Historia_Academica/UPaulSabatier/Scripts/KeggTask')
Entrez.email = "omalsa01@gmail.com"
#Function to search keys and return value
def findvalue(d, k):
    if k in d:
        var1 = d[k]
        return(var1)

In [2]:
#START KEGG INTERFACE
h = KEGG()
#DEFINE ORGANISM DATABASE. IN THIS CASE, HUMAN = hsa
h.organism= "hsa"

In [3]:
#OBTAIN THE PATHWAY IDS
f= open('Pathway_IDs.txt', 'r')
pway_ids= f.readlines()
#Initialize variables
kegg_ids= []
#GET THE KEGG IDS FOR THE GENES INVOLVED IN EACH PATHWAY
for i in pway_ids:
    tmp_ids= []
    #Name the pathway 
    query= 'hsa' + i
    #Get the data
    data= h.get(query)
    data= h.parse(data)
    genes= data['GENE']
    tmp_ids= genes.keys()
    #Concatenate them into a list with no repeats.
    kegg_ids.extend(tmp_ids)
    kegg_ids= list(set(kegg_ids))
#Print the number of genes involved in metabolism according to kegg
print(len(kegg_ids))

1633


In [4]:
#GET KEGG INFORMATION FOR EACH GENE
#Initialize dictionary
metabolism_data={}
c=0
#LOOP FOR ITERATE OVER THE KEGG IDS
for j in kegg_ids:
    print "Todo va bien", j, c
    query= 'hsa:' + j
    data= h.get(query)
    #Convert it it into a dictionary
    dict_data= h.parse(data)

    #PARSE BRITE INFO IN ORDER TO GET PATHWAYS AND REFERENCE PATHWAYS
    #The dict value is splited by line skip
    obj= dict_data['BRITE']
    obj= obj.split('\n')
    #Initialize variables
    pathw= []
    map_pathw={}
    genomic_info={}
    flag=0
    chr_num=0
    chr_loc=0
    exon_count=0
    assembly_acc=0
    annotation_release=0
    chr_acc=0
    chr_stop=0
    chr_start=0
    ref_pathw= None
    for i in obj:
        #Regular expressions to match r: reference and p: pathways
        r= re.search(r'(^\s{14}(\w.+))', i)
        p= re.search(r'\s+\d{5}\s(\w.+)', i)
        if r:
            #Evaluate if the reference is in the metabolism
            if r.group(2) in {"Carbohydrate metabolism", "Energy metabolism","Lipid metabolism", "Nucleotide metabolism"\
                     ,"Amino acid metabolism", "Metabolism of other amino acids","Glycan biosynthesis and metabolism"\
                     ,"Metabolism of cofactors and vitamins","Metabolism of terpenoids and polyketides"\
                     ,"Biosynthesis of other secondary metabolites","Xenobiotics biodegradation and metabolism"}:
                ref_pathw= r.group(2)
                flag=1
                pathw=[]
                #Flag changes in order to only obtain the genes involved in the reference
            else:
                flag=0
        if flag == 1:
            if p:
                #Get the pathways
                pathw.append(p.group(1))
    #Fill the dictionary. The reference pathways are the keys. Each pathway, then, is the value.
        if ref_pathw is None:
            continue
        map_pathw.update({ref_pathw: pathw})
        ref_pathw = None

    #GET THE GENOMIC INFORMATION 
    #SET THE PARAMETERS FOR ENTREZ SEARCH
    handle = Entrez.esummary(db="gene", id=j, rettype="gbwithparts", retmode="text", term="9606[Orgn]")
    record = Entrez.read(handle)
    handle.close()
    #print "Query:", query, record["DocumentSummarySet"]['DocumentSummary'][0]['GenomicInfo'][0]['ChrLoc']
    #ACCESS THE DICTIONARY AND LISTS. GET THE INFORMATION OF INTEREST.
    if record["DocumentSummarySet"]['DocumentSummary'][0]['GenomicInfo']:
    #Chromosome number
        chr_num= record["DocumentSummarySet"]['DocumentSummary'][0]['GenomicInfo'][0]['ChrLoc']
    #Exon count
        exon_count= record["DocumentSummarySet"]['DocumentSummary'][0]['GenomicInfo'][0]['ExonCount']
    else:
        chr_num= "NULL"
    #Location in chromosome
    if record["DocumentSummarySet"]['DocumentSummary'][0]['MapLocation']:
        chr_loc= record["DocumentSummarySet"]['DocumentSummary'][0]['MapLocation']
    else:
        chr_loc= "NULL" 
    #Assembly accession id
    assembly_acc= record["DocumentSummarySet"]['DocumentSummary'][0]['LocationHist'][0]['AssemblyAccVer']
    #Annotation id 
    annotation_release= record["DocumentSummarySet"]['DocumentSummary'][0]['LocationHist'][0]['AnnotationRelease']
    #Chromosome coordinates
    chr_acc= record["DocumentSummarySet"]['DocumentSummary'][0]['LocationHist'][0]['ChrAccVer']
    chr_stop= record["DocumentSummarySet"]['DocumentSummary'][0]['LocationHist'][0]['ChrStop']
    chr_start= record["DocumentSummarySet"]['DocumentSummary'][0]['LocationHist'][0]['ChrStart']
    #Create a dictionary for genomic info
    genomic_info= {'CHR_LOC': chr_loc, 'CHR_NUM': chr_num, 'EXON_COUNT': exon_count, 'ASSEMBLY': assembly_acc, 'CHR_ACC': chr_acc,
                   'CHR_STOP': chr_stop, 'CHR_START': chr_start, 'NCBIid': dict_data['DBLINKS']['NCBI-GeneID'],
                   'NAME': record["DocumentSummarySet"]['DocumentSummary'][0]['Name']}
    info_list= (genomic_info, map_pathw)
    #CONSTRUCT THE DICTIONARY WITH THE CRITERIA REQUESTED
    metabolism_data.update({j: info_list})
    c+=1

Todo va bien 55349 0
Todo va bien 55613 1
Todo va bien 4023 2
Todo va bien 51268 3
Todo va bien 5986 4
Todo va bien 3992 5
Todo va bien 3990 6
Todo va bien 8803 7
Todo va bien 1158 8
Todo va bien 55902 9
Todo va bien 55907 10
Todo va bien 55904 11
Todo va bien 93432 12
Todo va bien 4731 13
Todo va bien 81537 14
Todo va bien 55586 15
Todo va bien 51604 16
Todo va bien 290 17
Todo va bien 51606 18
Todo va bien 4524 19
Todo va bien 56052 20
Todo va bien 346606 21
Todo va bien 270 22
Todo va bien 271 23
Todo va bien 272 24
Todo va bien 275 25
Todo va bien 276 26
Todo va bien 277 27
Todo va bien 278 28
Todo va bien 279 29
Todo va bien 283871 30
Todo va bien 80201 31
Todo va bien 253558 32
Todo va bien 51111 33
Todo va bien 200895 34
Todo va bien 122618 35
Todo va bien 9524 36
Todo va bien 108 37
Todo va bien 109 38
Todo va bien 56901 39
Todo va bien 100 40
Todo va bien 107 41
Todo va bien 2517 42
Todo va bien 79813 43
Todo va bien 79814 44
Todo va bien 64902 45
Todo va bien 9415 46
Todo va 

In [14]:
#SAVE DATABASE INTO A TXT FILE. Pickle is used to mantain the data structure. 
with open('metabolism_pythondict.txt', 'wb') as handle:
  pickle.dump(metabolism_data, handle)

In [6]:
#ASK FOR DATABASES VERSION
handle = Entrez.einfo(db="gene", term="9606[Orgn]")
record = Entrez.read(handle)
record["DbInfo"]["Description"]
record["DbInfo"]["LastUpdate"]
#LOOK FOR THE KEYS
#for k in record["DocumentSummarySet"]['DocumentSummary'][0].keys():
#    print k
#TEST INFORMATION
handle = Entrez.esummary(db="gene", id="3101", rettype="gbwithparts", retmode="text", term="9606[Orgn]")
record = Entrez.read(handle)
handle.close()
print(record['DocumentSummarySet']['DocumentSummary'][0]['GenomicInfo'])
print(record["DocumentSummarySet"]['DocumentSummary'][0]['LocationHist'][0:3])

[{u'ChrAccVer': 'NC_000005.10', u'ChrLoc': '5', u'ExonCount': '22', u'ChrStop': '176880868', u'ChrStart': '176899372'}]
[{u'AssemblyAccVer': 'GCF_000001405.33', u'ChrAccVer': 'NC_000005.10', u'AnnotationRelease': '108', u'ChrStop': '176880868', u'ChrStart': '176899372'}, {u'AssemblyAccVer': 'GCF_000306695.2', u'ChrAccVer': 'NC_018916.2', u'AnnotationRelease': '108', u'ChrStop': '175740077', u'ChrStart': '175758540'}, {u'AssemblyAccVer': 'GCF_000001405.28', u'ChrAccVer': 'NC_000005.10', u'AnnotationRelease': '107', u'ChrStop': '176880868', u'ChrStart': '176899372'}]
